<a href="https://colab.research.google.com/github/imthelizardking/cmp719-project/blob/main/cmp719_project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Main text

Mount gdrive for saving weights etc.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
!cd '/content/drive/MyDrive'

Install torchvision package

In [20]:
!pip install torch torchvision

Import required packages

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
#from torchvision.models import resnet

ResNet-56 Model:

In [22]:
# Define the ResNet-56 model
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        return out

class ResNet56(nn.Module):
    def __init__(self, num_classes):
        super(ResNet56, self).__init__()
        self.in_planes = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 9, stride=1)
        self.layer2 = self._make_layer(32, 9, stride=2)
        self.layer3 = self._make_layer(64, 9, stride=2)
        self.linear = nn.Linear(64, num_classes)

    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes * BasicBlock.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = nn.ReLU()(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = nn.AdaptiveAvgPool2d(1)(out)
        out = torch.flatten(out, 1)
        out = self.linear(out)
        return out

Set configuration for training ResNet-56 w/ cifar-100:

In [31]:


# Set device to GPU if available, otherwise use CPU
device = torch.device("cuda")

# Define transforms for data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load CIFAR-100 dataset
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

# Define the ResNet-56 model
model = ResNet56(num_classes=100).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)



Files already downloaded and verified
Files already downloaded and verified


Train ResNet-56 w/ cifar-100

In [32]:
# Training loop
for epoch in range(100):  # Number of epochs
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 100 == 99:  # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print('Accuracy on test set after epoch %d: %.2f %%' % (epoch + 1, accuracy))

print('Training finished.')

[1,   100] loss: 4.595
[1,   200] loss: 4.507
[1,   300] loss: 4.381
Accuracy on test set after epoch 1: 2.88 %
[2,   100] loss: 4.258
[2,   200] loss: 4.244
[2,   300] loss: 4.202
Accuracy on test set after epoch 2: 4.18 %
[3,   100] loss: 4.154
[3,   200] loss: 4.135
[3,   300] loss: 4.112
Accuracy on test set after epoch 3: 4.10 %
[4,   100] loss: 4.071
[4,   200] loss: 4.043
[4,   300] loss: 4.035
Accuracy on test set after epoch 4: 5.83 %
[5,   100] loss: 4.006
[5,   200] loss: 3.993
[5,   300] loss: 3.965
Accuracy on test set after epoch 5: 8.21 %
[6,   100] loss: 3.950
[6,   200] loss: 3.995
[6,   300] loss: 4.013
Accuracy on test set after epoch 6: 7.75 %
[7,   100] loss: 3.888
[7,   200] loss: 3.880
[7,   300] loss: 3.919
Accuracy on test set after epoch 7: 9.69 %
[8,   100] loss: 3.834
[8,   200] loss: 3.853
[8,   300] loss: 3.808
Accuracy on test set after epoch 8: 10.87 %
[9,   100] loss: 3.769
[9,   200] loss: 3.754
[9,   300] loss: 3.738
Accuracy on test set after epoch 9

Save trained ResNet-56 weights:

In [33]:
torch.save(model.state_dict(), '/content/drive/MyDrive/719_project/trained_weights/resnet_model_weights.pth')

Calculate Top-1 accuracy for trained ResNet-56 model and cifar-100 dataset (in paper, 70.43%):

In [ ]:
with torch.no_grad(): # for making sure there is no training, just inference
    model.eval()  # model to eval. mode
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

top1_accuracy = 100 * correct / total
print('Top-1 Accuracy: {:.2f}%'.format(top1_accuracy))

# **Vision Transformer w/ Feature Guidance:**

Vision Transformer w/ feature guidance:

In [ ]:
# Define the Vision Transformer model
class VisionTransformer_fg(nn.Module):
    def __init__(self, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim):
        super(VisionTransformer_fg, self).__init__()
        num_patches = (image_size // patch_size) ** 2
        patch_dim = 3 * patch_size ** 2  # Assuming RGB images

        self.patch_embedding = nn.Sequential(
            nn.Conv2d(3, dim, kernel_size=patch_size, stride=patch_size),
            nn.Flatten(start_dim=2)
        )
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(dim, heads, dim_feedforward=mlp_dim),
            num_layers=depth
        )
        self.classifier = nn.Linear(dim, num_classes)

    def forward(self, x):
        x = self.patch_embedding(x)
        x = x.permute(1, 0, 2)
        x = self.transformer(x)
        x = x.mean(dim=0)
        x = self.classifier(x)
        return x

Trainer:

In [ ]:
# Define the training loop
def train_ViT_with_fg(model_ViT, model_cnn, dataloader, criterion, optimizer, device):
    model_cnn.eval() # no training for cnn, just eval.
    model_ViT.train()
    total_loss = 0.0

    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model_ViT(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * images.size(0)

    return total_loss / len(dataloader.dataset)

Set training configuration for Vision Transformer w/ feature guidance

In [ ]:
# Set up hyperparameters and data loaders
image_size = 224
patch_size = 16
num_classes = 10
dim = 256
depth = 12
heads = 8
mlp_dim = 512
batch_size = 32
learning_rate = 1e-3
epochs = 10

# Create an instance of the model
model = VisionTransformer_fg(image_size, patch_size, num_classes, dim, depth, heads, mlp_dim)
model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create data loaders (replace with your own datasets)
train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
train_dataloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

Vision Transformer Trainer:

In [ ]:
# Training loop
for epoch in range(epochs):
    train_loss = train_ViT_with_fg(model, model, train_dataloader, criterion, optimizer, device)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {train_loss:.4f}")